# Лекция 8: Инжиниринг признаков и Логистическая регрессия

**Цели лекции:**
1.  Сформировать понимание важности и основных техник инжиниринга признаков.
2.  Научиться работать с отсутствующими данными, выбросами и категориальными переменными.
3.  Понять, почему линейная регрессия не подходит для задач классификации.
4.  Изучить математические основы логистической регрессии, включая логистическую функцию, шансы и log-odds.
5.  Освоить ключевые метрики оценки качества моделей классификации (Confusion Matrix, Accuracy, Precision, Recall, F1, ROC/AUC).
6.  Рассмотреть расширение логистической регрессии на случай нескольких классов.

## Часть 1: Инжиниринг признаков (Feature Engineering)

**Инжиниринг признаков (Feature Engineering)** — это процесс использования знаний о предметной области для создания новых признаков из уже существующих. Цель — представить данные в таком виде, который наилучшим образом описывает базовую проблему для алгоритмов машинного обучения. Качество модели на 80% зависит от качества признаков, а не от сложности самого алгоритма.

#### Три основных подхода в инжиниринге признаков:

1.  **Извлечение (Extraction):** Создание новых, более простых признаков из сложных. Например, извлечение дня недели, месяца или времени суток из полной временной метки `timestamp`.
2.  **Комбинирование (Combination):** Объединение нескольких признаков в один. Например, создание полиномиальных признаков (`TV_budget * Radio_budget`) для учета их совместного влияния.
3.  **Преобразование (Transformation):** Изменение исходных признаков для улучшения их свойств. К этому относится масштабирование (`StandardScaler`), логарифмирование или кодирование категориальных признаков, которое мы рассмотрим ниже.

### 1.1. Работа с отсутствующими данными (Missing Data)

Реальные данные почти никогда не бывают полными. Пропуски могут возникать из-за ошибок сбора данных, сбоев систем или потому, что какой-то признак просто неприменим к объекту (например, 'год постройки гаража' для дома без гаража).

In [ ]:
import pandas as pd
import numpy as np

# Создадим простой DataFrame с пропусками
data = {'temperature': [25, 26, np.nan, 28, 29, 24],
        'humidity': [80, np.nan, 82, 83, 81, 79],
        'wind_speed': [10, 12, 11, np.nan, 13, 9]}
sample_df = pd.DataFrame(data)

print("Исходные данные с пропусками:")
print(sample_df)

print("\nПоиск пропусков с помощью .isnull().sum():")
print(sample_df.isnull().sum())

#### 1.1.1. Удаление данных

Самый простой подход. Можно удалять либо строки (`.dropna(axis=0)`), либо целые столбцы (`.dropna(axis=1)`). Это оправдано, если пропусков очень мало, иначе мы рискуем потерять ценную информацию.

In [ ]:
# Удаление строк с любым пропущенным значением
print("DataFrame после удаления строк:")
print(sample_df.dropna())

#### 1.1.2. Заполнение (импутация) данных

Более предпочтительный метод. Пропуски можно заполнять:
*   **Простыми значениями:** нулем, средним (`.mean()`), медианой (`.median()`) или модой (самым частым значением).
*   **Продвинутыми методами:** например, предсказывая пропущенное значение на основе других признаков.

In [ ]:
# Заполнение пропусков средним значением по столбцу
print("DataFrame после заполнения средним:")
print(sample_df.fillna(sample_df.mean()))

### 1.2. Работа с дублями (Duplicates)

**Дубликаты** — это строки в наборе данных, которые являются полностью идентичными. Наличие дублей может привести к нескольким проблемам:

1.  **Искажение результатов:** Дубликаты придают неоправданно больший вес определенным наблюдениям, что может сместить статистические показатели (среднее, медиану) и повлиять на обучение модели.
2.  **Утечка данных (Data Leakage):** Самая серьезная проблема. Если одна и та же строка попадет и в обучающую, и в тестовую выборку, модель получит "бесплатный" правильный ответ на тестовых данных. Это приведет к искусственно завышенным метрикам качества и ложному представлению о том, как модель будет работать на действительно новых данных.

Поэтому **первым шагом** в любом проекте по анализу данных должна быть проверка и удаление полных дубликатов.

**Как с ними работать в Pandas:**
*   **Обнаружение:** Метод `.duplicated().sum()` позволяет быстро посчитать количество полных дубликатов.
*   **Просмотр:** Чтобы увидеть все дублирующиеся строки (включая их "оригиналы"), используется `df[df.duplicated(keep=False)]`.
*   **Удаление:** Метод `.drop_duplicates()` удаляет дубликаты, по умолчанию оставляя первое вхождение каждой строки.

In [ ]:
# Создадим DataFrame с явными дубликатами
data = {'Имя': ['Арман', 'Айгерим', 'Бауыржан', 'Арман', 'Алия', 'Айгерим'],
        'Возраст': [25, 30, 35, 25, 28, 30],
        'Город': ['Астана', 'Алматы', 'Шымкент', 'Астана', 'Караганда', 'Алматы']}
df_duplicates = pd.DataFrame(data)

print("Исходный DataFrame:")
print(df_duplicates)

# --- Обнаружение дублей ---
print("\n--- Обнаружение ---")
num_duplicates = df_duplicates.duplicated().sum()
print(f"Количество полных дубликатов в данных: {num_duplicates}")

# Посмотрим на сами дублирующиеся строки
print("\nОтображение всех дублирующихся строк (включая оригиналы):")
print(df_duplicates[df_duplicates.duplicated(keep=False)])


# --- Удаление дублей ---
print("\n--- Удаление ---")
df_cleaned = df_duplicates.drop_duplicates()
print("DataFrame после удаления дублей:")
print(df_cleaned)

# --- Проверка ---
print("\n--- Проверка ---")
print(f"Количество дубликатов после очистки: {df_cleaned.duplicated().sum()}")

### 1.3. Работа с выбросами (Outliers)

**Выброс** — это точка данных, которая сильно отличается от остальных. Выбросы могут искажать результаты обучения, "перетягивая" на себя линию регрессии.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Сгенерируем нормальное распределение и добавим выбросы
np.random.seed(42)
data_normal = np.random.normal(loc=100, scale=10, size=100)
data_with_outliers = np.concatenate([data_normal, [180, 190, -50]]) # Добавляем 3 выброса
df_outliers = pd.DataFrame(data_with_outliers, columns=['value'])

# Визуализируем с помощью boxplot
plt.figure(figsize=(10, 5))
sns.boxplot(data=df_outliers, x='value')
plt.title('Обнаружение выбросов с помощью Box Plot')
plt.show()

#### Практический пример: Ограничение выбросов по методу межквартильного размаха (IQR)

Этот метод является статистическим аналогом "усов" на boxplot. Выбросами считаются все точки, которые лежат за пределами:

<br>
$$ Нижняя\_граница = Q1 - 1.5 \cdot IQR $$
<br>
$$ Верхняя\_граница = Q3 + 1.5 \cdot IQR $$
<br>

где $Q1$ — 25-й перцентиль, $Q3$ — 75-й перцентиль, $IQR = Q3 - Q1$. Вместо удаления, мы можем "ограничить" значения, заменив все, что выходит за рамки, на значения границ.

In [ ]:
# Рассчитаем границы
Q1 = df_outliers['value'].quantile(0.25)
Q3 = df_outliers['value'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Нижняя граница: {lower_bound:.2f}")
print(f"Верхняя граница: {upper_bound:.2f}")

# Ограничим значения (clipping)
df_clipped = df_outliers.copy()
df_clipped['value'] = df_clipped['value'].clip(lower=lower_bound, upper=upper_bound)

# Визуализируем результат
plt.figure(figsize=(10, 5))
sns.boxplot(data=df_clipped, x='value')
plt.title('Данные после ограничения выбросов')
plt.show()

### 1.4. Работа с категориальными данными

**Категориальные данные** — это переменные, которые содержат метки, а не числовые значения. Они описывают принадлежность объекта к какой-либо группе.

Существует два основных типа:
1.  **Номинальные (Nominal):** Категории не имеют внутреннего порядка. *Примеры: 'Город' (Москва, Казань), 'Пол' (Мужской, Женский).* 
2.  **Порядковые (Ordinal):** Категории имеют естественный порядок или ранг. *Примеры: 'Размер' (S, M, L), 'Оценка' (Плохо, Хорошо, Отлично).* 

Большинство моделей машинного обучения работают с числами, поэтому текстовые категории нужно преобразовать.

#### 1.4.0. Кодирование чисел (Integer/Label Encoding)

Это самый простой способ преобразования: каждой уникальной категории присваивается целое число (0, 1, 2 и так далее).

**Однако, этот метод нужно использовать с большой осторожностью!**

**Проблема: Ложная упорядоченность**

Когда мы присваиваем числа `1`, `2`, `3` категориям, большинство моделей машинного обучения воспримут их как упорядоченные величины. Они "подумают", что `3 > 2 > 1`.

*   **Когда это плохо (для номинальных данных):** Если мы кодируем страны: `{'США': 1, 'Мексика': 2, 'Канада': 3}`, модель может ошибочно сделать вывод, что "Канада" в каком-то смысле "больше" или "важнее", чем "Мексика". Это вносит в данные ложную информацию, которая может навредить качеству модели.

*   **Когда это хорошо (для порядковых данных):** Если наши категории имеют естественный, логический порядок (например, уровень остроты блюда), то кодирование числами становится правильным выбором. В этом случае порядок `{'Mild': 1, 'Hot': 2, 'Fire': 3}` несет полезную информацию для модели.

**Вывод:** Используйте кодирование числами только для **порядковых (ordinal)** признаков. Для **номинальных (nominal)** признаков этот метод, как правило, вреден.

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-14.png" alt="Кодирование стран" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-16.png" alt="Кодирование остроты" width="400"></td>
  </tr>
</table>

#### 1.4.1. Преобразование непрерывных данных в категории (Binning)

Иногда полезно превратить непрерывный признак (например, возраст) в категориальный. Этот процесс называется **дискретизацией** или **биннингом** (от англ. *bin* — корзина). Это может помочь модели уловить нелинейные зависимости, которые она не смогла бы обнаружить в исходных данных. Например, влияние возраста на какой-либо показатель может быть нелинейным: сначала оно растет, а после определенной точки начинает снижаться. Создание возрастных групп ('Молодой', 'Взрослый', 'Пожилой') позволяет модели присвоить каждой группе свой собственный, независимый вес.

In [ ]:
# Создадим серию с разными возрастами
ages = pd.Series([15, 22, 35, 48, 65, 70, 28, 55])

# Определим границы для возрастных групп (бины)
# [0, 18) -> 0-17 лет
# [18, 35) -> 18-34 лет
# [35, 60) -> 35-59 лет
# [60, 100) -> 60-99 лет
bins = [0, 18, 35, 60, 100]

# Определим названия для этих групп
labels = ['Ребенок', 'Молодой', 'Взрослый', 'Пожилой']

# Используем функцию pd.cut для разбиения данных на категории
age_groups = pd.cut(ages, bins=bins, labels=labels, right=False)

print("Исходный возраст:")
print(ages)
print("\nПосле разбиения на категории (Binning):")
print(age_groups)

#### 1.4.2. Преобразование числовых кодов в категории (Type Conversion)

Очень часто в реальных данных категориальные признаки уже закодированы числами. Например, в датасете Ames Housing признак `MSSubClass` (тип жилья) представлен числами: 20, 30, 60 и т.д. 

**Проблема:** Если оставить эти данные как есть, большинство алгоритмов машинного обучения (особенно линейные модели) будут ошибочно интерпретировать их как непрерывные числовые переменные. Модель может предположить, что между классами есть математическая зависимость (например, что класс 60 в 3 раза 'больше' или 'важнее' класса 20), хотя на самом деле это просто уникальные коды.

**Решение:** Чтобы избежать этого, необходимо явно указать Pandas, что этот столбец следует рассматривать как категориальный. Самый надежный способ — преобразовать тип данных этого столбца в `object` или `str` **перед** применением One-Hot Encoding. После такого преобразования функция `pd.get_dummies` корректно распознает каждое число как отдельную категорию и создаст для него свой бинарный столбец.

In [ ]:
# Создадим DataFrame, имитирующий проблему
subclass_df = pd.DataFrame({'MSSubClass': [20, 30, 60, 20, 70]})

print("Исходный DataFrame и тип данных:")
print(subclass_df)
print(f"Тип данных столбца: {subclass_df['MSSubClass'].dtype}")

# --- ПРАВИЛЬНЫЙ ПОДХОД ---

# Шаг 1: Преобразуем тип данных в 'object' (или 'str')
subclass_df['MSSubClass'] = subclass_df['MSSubClass'].astype(str)

print("\nТип данных после преобразования:")
print(f"Новый тип данных столбца: {subclass_df['MSSubClass'].dtype}")

# Шаг 2: Теперь применяем One-Hot Encoding
dummies = pd.get_dummies(subclass_df, drop_first=True)

print("\nРезультат One-Hot Encoding после правильного преобразования типов:")
print(dummies)

#### 1.4.3. One-Hot Encoding для номинальных признаков

Для **номинальных** признаков (где нет порядка) используется **One-Hot Encoding**. Он создает новый бинарный столбец (0 или 1) для каждой категории. Это стандартный и наиболее безопасный способ кодирования таких данных.

![OHE](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-19.png)

In [ ]:
# Пример с городами
cities = pd.Series(['Almaty', 'Astana', 'Almaty', 'Astana', 'Shymkent'])

print("Исходные данные:")
print(cities)

print("\nПосле One-Hot Encoding (с удалением первого столбца):")
# drop_first=True, чтобы избежать ловушки фиктивных переменных
print(pd.get_dummies(cities, drop_first=True))

## Часть 2: Логистическая регрессия

Прежде чем изучать конкретный алгоритм, давайте четко определим, какую задачу мы собираемся решать.

### Задача классификации

**Классификация (Classification)** — это одна из основных задач машинного обучения с учителем. Ее цель — предсказать, к какому из нескольких предопределенных **классов** (категорий) принадлежит объект. В отличие от задачи регрессии, где мы предсказывали непрерывное число (например, цену дома), здесь мы предсказываем дискретную метку.

*   **Регрессия отвечает на вопрос "Сколько?":** *Сколько стоит квартира? Какая завтра будет температура?*
*   **Классификация отвечает на вопрос "Какой?":** *Это письмо — спам или нет? Какое животное на картинке?*

### Типы задач классификации

Задачи классификации делятся на два основных типа:

1.  **Бинарная классификация (Binary Classification):**
    *   **Что это:** Задача, в которой существует ровно два взаимоисключающих класса. Это самый распространенный тип классификации.
    *   **Примеры:**
        *   Медицинский диагноз: есть заболевание (`1`) или нет (`0`).
        *   Спам-фильтр: письмо является спамом (`1`) или нет (`0`).
        *   Банковский скоринг: клиент вернет кредит (`1`) или не вернет (`0`).
        *   Маркетинг: пользователь кликнет на рекламу (`1`) или нет (`0`).

2.  **Многоклассовая классификация (Multiclass Classification):**
    *   **Что это:** Задача, в которой существует более двух взаимоисключающих классов. Объект может принадлежать только к одному из них.
    *   **Примеры:**
        *   Распознавание рукописных цифр: `0`, `1`, `2`, ..., `9`.
        *   Классификация новостей по темам: 'Политика', 'Спорт', 'Технологии', 'Культура'.
        *   Анализ тональности текста: 'Позитивный', 'Нейтральный', 'Негативный'.
        *   Классификация цветков ириса (как мы увидим далее): 'Setosa', 'Versicolor', 'Virginica'.

### Логистическая регрессия: Основной инструмент для бинарной классификации

Фундаментальным и наиболее часто используемым алгоритмом для решения задач **бинарной классификации** является **логистическая регрессия**. Несмотря на слово "регрессия" в названии, это именно метод классификации, который предсказывает вероятность принадлежности объекта к одному из двух классов.

Хотя ее основное предназначение — это бинарная классификация, существуют методы (например, One-vs-Rest), которые позволяют расширить ее и для решения многоклассовых задач, что мы рассмотрим на примере датасета Ирисов Фишера.

### 2.1. От регрессии к классификации

Представим, что у нас есть данные о студентах (часы подготовки) и результат экзамена (сдал/не сдал). Линейная регрессия здесь не подойдет, так как ее предсказания могут выходить за пределы [0, 1].

![linear-to-logitic](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-37.png)

In [ ]:
from sklearn.linear_model import LinearRegression

# Синтетические данные
hours = np.array([0.5, 0.75, 1, 1.25, 1.5, 1.75, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 4, 4.25, 4.5, 4.75, 5, 5.5]).reshape(-1, 1)
passed = np.array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1])

lin_reg = LinearRegression()
lin_reg.fit(hours, passed)

plt.figure(figsize=(10, 6))
plt.scatter(hours, passed, color='blue', label='Данные')
plt.plot(hours, lin_reg.predict(hours), color='red', label='Линейная регрессия')
plt.axhline(y=0, color='grey', linestyle='--')
plt.axhline(y=1, color='grey', linestyle='--')
plt.title('Почему линейная регрессия не подходит для классификации')
plt.xlabel('Часы подготовки')
plt.ylabel('Результат (0 - не сдал, 1 - сдал)')
plt.legend()
plt.show()

### 2.2. Логистическая функция (Сигмоида)

Решение — пропустить выход линейной модели через S-образную **сигмоиду**, которая преобразует любое число в вероятность от 0 до 1.

<br>
$$ \sigma(z) = \frac{1}{1 + e^{-z}} $$
<br>

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

z = np.linspace(-10, 10, 100)
plt.figure(figsize=(10, 4))
plt.plot(z, sigmoid(z))
plt.title('График сигмоидной функции')
plt.xlabel('z')
plt.ylabel('$\sigma(z)$')
plt.grid(True)
plt.show()

Пример: income = 1. Вероятность возврата кредита - 90%

![ExmplPossibility](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-41.png)

### 2.3. Уравнение логистической регрессии

Мы подставляем линейную часть $z = w_0 + w_1x_1 + ...$ в сигмоиду. Модель предсказывает вероятность принадлежности к классу '1'.

![Формула](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-48.png)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(hours, passed)

# Создаем плавную кривую для графика
x_test = np.linspace(0, 6, 100).reshape(-1, 1)
y_prob = log_reg.predict_proba(x_test)[:, 1] # Вероятность класса 1

plt.figure(figsize=(10, 6))
plt.scatter(hours, passed, color='blue', label='Данные')
plt.plot(x_test, y_prob, color='green', label='Логистическая регрессия')
plt.title('Логистическая регрессия для классификации')
plt.xlabel('Часы подготовки')
plt.ylabel('Вероятность сдачи экзамена')
plt.legend()
plt.show()

2.3.1. Интерпретация: от Вероятностей к Log-Odds и обратно

В отличие от линейной регрессии, коэффициенты логистической регрессии $w_i$ нельзя интерпретировать напрямую как "изменение $y$ при изменении $x_i$ на единицу". Они влияют на результат нелинейно через сигмоиду. Чтобы понять их смысл, нужно ввести понятия **Шансы (Odds)** и **Логарифм шансов (Log-Odds)**.

**Шаг 1: От Вероятности к Шансам (Odds)**

Шансы — это отношение вероятности того, что событие произойдет, к вероятности того, что оно не произойдет. Если вероятность сдать экзамен ($p$) равна 0.8, то вероятность не сдать ($1-p$) равна 0.2. Шансы сдать равны 0.8 / 0.2 = 4, или "4 к 1".

<br>
$$ Odds = \frac{p}{1 - p} $$
<br>


**Шаг 2: От Шансов к Логарифму Шансов (Log-Odds)**

Взяв натуральный логарифм от шансов, мы получаем величину, которая изменяется от $-\infty$ до $+\infty$. Это и есть **Log-Odds**.

<br>
$$ Log\_Odds = \ln\left(\frac{p}{1 - p}\right) $$
<br>

**Ключевая идея логистической регрессии:** Линейная комбинация признаков $z = w_0 + w_1x_1 + ... + w_mx_m$ на самом деле является моделью не для самой вероятности, а для **логарифма шансов**.

<br>
$$ \ln\left(\frac{p}{1 - p}\right) = w_0 + w_1x_1 + ... + w_mx_m $$
<br>

Это означает, что при увеличении признака $x_i$ на одну единицу, **логарифм шансов** увеличивается на $w_i$.


![logodds](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-71.png)

**Шаг 3: Обратный путь — от Log-Odds к Вероятности**

Как, зная предсказанный моделью Log-Odds ($z$), получить вероятность ($p$)? Давайте выразим $p$ из уравнения выше:

1. Возьмем экспоненту от обеих частей: $$ \frac{p}{1 - p} = e^z $$
2. Домножим на $(1-p)$: $$ p = e^z \cdot (1-p) $$
3. Раскроем скобки: $$ p = e^z - p \cdot e^z $$
4. Перенесем все слагаемые с $p$ налево: $$ p + p \cdot e^z = e^z $$
5. Вынесем $p$ за скобки: $$ p(1 + e^z) = e^z $$
6. Получим финальную формулу: $$ p = \frac{e^z}{1 + e^z} $$

Если разделить числитель и знаменатель на $e^z$, мы получим уже знакомую нам формулу сигмоиды:

<br>
$$ p = \frac{e^z/e^z}{(1+e^z)/e^z} = \frac{1}{1/e^z + 1} = \frac{1}{1 + e^{-z}} = \sigma(z) $$
<br>

Таким образом, **сигмоида — это просто математический способ перейти от предсказанных моделью Log-Odds обратно к вероятности.**

### 2.4. Обучение модели: Метод максимального правдоподобия и Log Loss

Как модель находит оптимальные веса `w`? В отличие от линейной регрессии, где мы минимизировали сумму квадратов ошибок (MSE), в логистической регрессии используется **Метод максимального правдоподобия (Maximum Likelihood Estimation, MLE)**.

![whatisthebest](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-77.png)


**Интуиция на простом примере:**
Представьте, что вы подбросили монетку 10 раз и получили 7 орлов и 3 решки. Какова, по-вашему, вероятность выпадения орла ($p$)? Большинство скажет, что 0.7. Вы интуитивно использовали MLE.

**Логика MLE:** Давайте найдем такое значение параметра ($p$), при котором вероятность получить **именно те данные, которые мы наблюдали**, будет максимальной.
*   Если бы $p=0.5$ (честная монета), вероятность нашей последовательности была бы $(0.5)^7 \cdot (1-0.5)^3 \approx 0.00097$.
*   Если бы $p=0.7$, вероятность нашей последовательности была бы $(0.7)^7 \cdot (1-0.7)^3 \approx 0.00222$.

Вероятность выше при $p=0.7$. MLE — это процесс нахождения такого $p$, который максимизирует эту вероятность (правдоподобие).

**Применительно к логистической регрессии:** Алгоритм подбирает такие веса `w`, которые максимизируют правдоподобие того, что для каждого объекта из обучающей выборки модель выдаст вероятность, близкую к его истинной метке (близкую к 1 для класса '1' и близкую к 0 для класса '0').

Математически, максимизация правдоподобия эквивалентна минимизации отрицательного логарифма правдоподобия. Эта функция потерь для логистической регрессии называется **Log Loss** (или бинарной кросс-энтропией).

Вот как выглядит эта функция для **одного** объекта:

<br>
$$ Loss(y, \hat{p}) = -[y \cdot \log(\hat{p}) + (1 - y) \cdot \log(1 - \hat{p})] $$
<br>

где:
*   $y$ — истинная метка класса (0 или 1).
*   $\hat{p}$ — предсказанная моделью вероятность того, что объект принадлежит к классу 1.

**Давайте разберемся, как она работает:**

*   **Случай 1: Истинная метка y = 1.**
    Формула упрощается до $Loss(1, \hat{p}) = -[1 \cdot \log(\hat{p}) + (0) \cdot \log(1 - \hat{p})] = -\log(\hat{p})$.
    Чтобы минимизировать эту потерю, значение $-\log(\hat{p})$ должно быть как можно меньше. Это происходит, когда $\log(\hat{p})$ максимален, то есть когда $\hat{p}$ стремится к **1**. Это именно то, что нам нужно!

*   **Случай 2: Истинная метка y = 0.**
    Формула упрощается до $Loss(0, \hat{p}) = -[0 \cdot \log(\hat{p}) + (1) \cdot \log(1 - \hat{p})] = -\log(1 - \hat{p})$.
    Чтобы минимизировать эту потерю, значение $-\log(1 - \hat{p})$ должно быть как можно меньше. Это происходит, когда $\log(1 - \hat{p})$ максимален, то есть когда $1 - \hat{p}$ стремится к 1, а само $\hat{p}$ — к **0**. Снова, именно то, что нам нужно!

Чтобы получить общую **функцию стоимости (Cost Function)** для всей обучающей выборки из N объектов, мы просто усредняем эту потерю:

<br>
$$ J(w) = -\frac{1}{N} \sum_{i=1}^{N} [y_i \cdot \log(\hat{p}_i) + (1 - y_i) \cdot \log(1 - \hat{p}_i)] $$
<br>

Поскольку предсказанная вероятность $\hat{p}_i$ является результатом применения сигмоидной функции к линейной комбинации признаков ($\hat{p}_i = \sigma(w^T x_i)$), то полная формула функции стоимости, которую необходимо минимизировать, выглядит следующим образом:

<br>
$$ J(w) = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \cdot \log\left( \frac{1}{1 + e^{-w^T x_i}} \right) + (1 - y_i) \cdot \log\left(1 - \frac{1}{1 + e^{-w^T x_i}}\right) \right] $$
<br>

Хотя эта формула выглядит сложной, она является выпуклой, что гарантирует наличие единственного глобального минимума. Как и в случае с линейной регрессией, для нахождения этого минимума (то есть оптимальных весов `w`) используется **метод градиентного спуска**.

![logodds-p](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-100.png)

### 2.5. Мультиклассовая Логистическая Регрессия

Что делать, если у нас больше двух классов? Стандартный подход называется **One-vs-Rest (OvR)**. Для K классов строится K бинарных классификаторов: первый отличает класс 1 от всех остальных, второй — класс 2 от всех остальных, и так далее. Для нового объекта запускаются все K классификаторов, и выбирается тот, который выдал наибольшую уверенность (вероятность).

Рассмотрим это на классическом датасете **Ирисы Фишера**.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X_iris = iris.data
y_iris = iris.target

df_iris = pd.DataFrame(X_iris, columns=iris.feature_names)
df_iris['species'] = y_iris

# Заменим числовые метки на названия для наглядности
target_names = iris.target_names
df_iris['species'] = df_iris['species'].apply(lambda x: target_names[x])

print("Первые 5 строк датасета Ирисов Фишера:")
print(df_iris.head())

sns.pairplot(df_iris, hue='species')
plt.show()

На `pairplot` видно, что класс `setosa` легко отделим, а `versicolor` и `virginica` частично пересекаются. Обучим модель, чтобы разделить все три класса.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# Подготовка данных
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

scaler_iris = StandardScaler()
X_train_iris_scaled = scaler_iris.fit_transform(X_train_iris)
X_test_iris_scaled = scaler_iris.transform(X_test_iris)

# Обучение модели. Scikit-learn автоматически использует OvR для мультиклассовой задачи.
log_reg_multi = LogisticRegression()
log_reg_multi.fit(X_train_iris_scaled, y_train_iris)

# Оценка
y_pred_iris = log_reg_multi.predict(X_test_iris_scaled)

print("Матрица ошибок для Ирисов Фишера:")
print(confusion_matrix(y_test_iris, y_pred_iris))
print("\nОтчет о классификации:")
print(classification_report(y_test_iris, y_pred_iris, target_names=target_names))

## Часть 3: Метрики оценки моделей классификации

Метрики для регрессии (MAE, RMSE) здесь не подходят. Для классификации используется свой набор метрик, основанный на **Матрице ошибок (Confusion Matrix)**.

### 3.1. Матрица ошибок (Confusion Matrix)

Это таблица, которая показывает, сколько предсказаний модель сделала правильно, а сколько — неправильно, для каждого класса. Она является основой для всех остальных метрик.

*   **True Positive (TP):** Истинно-положительные. Модель предсказала '1', и это был '1'.
*   **True Negative (TN):** Истинно-отрицательные. Модель предсказала '0', и это был '0'.
*   **False Positive (FP):** Ложно-положительные (Ошибка I рода). Модель предсказала '1', а на самом деле это был '0'.
*   **False Negative (FN):** Ложно-отрицательные (Ошибка II рода). Модель предсказала '0', а на самом деле это был '1'.

![confusionmatrix](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-126.png)

In [ ]:
from sklearn.metrics import confusion_matrix

# Примерные истинные значения и предсказания
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
y_pred = [1, 1, 1, 0, 0, 1, 0, 0, 1, 0]

cm = confusion_matrix(y_true, y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Класс 0', 'Класс 1'], yticklabels=['Класс 0', 'Класс 1'])
plt.xlabel('Предсказанные значения')
plt.ylabel('Истинные значения')
plt.title('Матрица ошибок')
plt.show()

### 3.2. Основные метрики

#### **Accuracy (Доля правильных ответов)**
*   **Вопрос:** Какова доля всех правильных предсказаний (и положительных, и отрицательных) в общем количестве наблюдений?
*   **Формула:** $$ Accuracy = \frac{TP+TN}{TP+TN+FP+FN} $$
*   **Диапазон:** от 0 до 1 (или от 0% до 100%).
    *   **Ближе к 1:** Модель делает много правильных предсказаний в целом.
    *   **Ближе к 0:** Модель часто ошибается.
*   **Интерпретация:** Простая и понятная метрика. Однако, она может вводить в заблуждение на **несбалансированных данных** — ситуациях, когда одного класса гораздо больше, чем другого. Это явление называется **Парадокс точности**.

**Пример Парадокса точности:**
Представим задачу диагностики редкого заболевания. В выборке 1000 человек: 990 здоровы (класс 0) и 10 больны (класс 1). Мы создали очень простую (и бесполезную) модель, которая всегда предсказывает "здоров" (класс 0).
*   **TN:** 990 (правильно угадала здоровых).
*   **TP:** 0 (не нашла ни одного больного).
*   **FP:** 0 (никого не назвала больным).
*   **FN:** 10 (пропустила всех больных).

Ее Accuracy будет: $$ Accuracy = \frac{0 + 990}{0 + 990 + 0 + 10} = \frac{990}{1000} = 99\\% $$. 
Точность 99% выглядит фантастически, но модель абсолютно бесполезна, так как не решает главную задачу — находить больных. Поэтому для несбалансированных данных `Accuracy` использовать нельзя. Нужны другие метрики.

![Precision1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-134.png)

![Precision2](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-137.png)

#### **Recall (Полнота, или Чувствительность)**
*   **Вопрос:** Из всех реальных 'положительных' объектов, какую долю мы смогли найти?
*   **Формула:** $$ Recall = \frac{TP}{TP + FN} $$
*   **Диапазон:** от 0 до 1.
    *   **Ближе к 1:** Модель находит почти все объекты положительного класса.
    *   **Ближе к 0:** Модель пропускает большинство объектов положительного класса.
*   **Интерпретация:** Recall важна, когда цена ошибки **False Negative** высока. *Пример: медицинская диагностика. Пропустить больного человека (FN) — катастрофа. Мы хотим, чтобы модель находила как можно больше реально больных, даже если при этом она иногда будет ошибочно отправлять здоровых на доп. обследование (FP). Здесь важен высокий Recall.* 

![Recall1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-146.png)

![Recall2](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-148.png)

#### **Precision (Точность)**
*   **Вопрос:** Из всех объектов, которые модель назвала 'положительными', какая доля действительно была 'положительной'?
*   **Формула:** $$ Precision = \frac{TP}{TP + FP} $$
*   **Диапазон:** от 0 до 1.
    *   **Ближе к 1:** Модель очень точна в своих положительных предсказаниях. Если она говорит "Да", то ей можно верить.
    *   **Ближе к 0:** Большинство положительных предсказаний модели — ложные тревоги.
*   **Интерпретация:** Precision важна, когда цена ошибки **False Positive** высока. *Пример: спам-фильтр. Если модель пометит важное письмо как спам (FP), это большая проблема. Мы хотим, чтобы модель была очень уверена в своих "спам"-вердиктах, то есть имела высокий Precision.* 

![Precision1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-150.png)

![Precision2](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-152.png)


#### **F1-Score**
*   **Вопрос:** Как найти баланс между Precision и Recall?
*   **Формула:** $$ F1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall} $$
*   **Диапазон:** от 0 до 1.
    *   **Ближе к 1:** У модели хороший баланс между точностью и полнотой.
    *   **Ближе к 0:** У модели низкая либо точность, либо полнота, либо и то, и другое.
*   **Интерпретация:** Гармоническое среднее между Precision и Recall. Эта метрика полезна, когда важны обе ошибки (FP и FN) и нужно найти компромисс между ними. Она наказывает модели, у которых одна из метрик (Precision или Recall) очень низкая.

In [ ]:
print(classification_report(y_true, y_pred))

### 3.3. ROC-кривая и AUC

Логистическая регрессия выдает вероятности. Чтобы получить классы (0 или 1), мы используем порог (обычно 0.5). **ROC-кривая** показывает качество модели при *всех возможных* пороговых значениях, строя график зависимости True Positive Rate (Recall) от False Positive Rate.

**AUC (Площадь под кривой)** — это интегральная метрика качества, не зависящая от порога. 
*   AUC = 1.0 — идеальный классификатор.
*   AUC = 0.5 — случайное угадывание.

Идеальная ROC & Реалистичная ROC

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-177.png" alt="Идеальная" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec8-162.png" alt="Реалистичная" width="400"></td>
  </tr>
</table>

In [ ]:
from sklearn.metrics import plot_roc_curve

# Используем модель, обученную на данных о подготовке к экзамену
plot_roc_curve(log_reg, hours, passed)
plt.plot([0, 1], [0, 1], 'k--', label='Случайное угадывание (AUC = 0.5)')
plt.title('ROC-кривая')
plt.legend()
plt.show()